<a href="https://colab.research.google.com/github/rgprado/computacao_nuvem/blob/main/ComputacaoEmNuvem_RodrigoPrado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sobre os dados

O arquivo CSV 'movies' contém o identificador e o nome do filme. O arquivo 'customers_rating' contém informações sobre ID do cliente, nota dada ao filme, qual a data da avaliação e o ID do filme.

**Descrição das colunas:**

Base 1:  
ID do filme  
título e ano de lançamento

Base 2:  
Cust_Id: ID do customer que fez a avaliação  
Rating: avaliação (nota)  
Date: data da avaliação  
Movie_Id: ID do filme  


**Amostra:**  
**movies:**  
1;(Dinosaur Planet, 2003)  
2;(Isle of Man TT 2004 Review, 2004)  
3;(Character, 1997)


**customers_rating:**  
1488844;3.0;2005-09-06;1  
822109;5.0;2005-05-13;1  
885013;4.0;2005-10-19;1

**Nome do arquivo CSV:**  
data/movies.csv  
data/customers_rating.csv

## Sobre as questões
Base1 - Movies: https://drive.google.com/file/d/1gLsCjaMrL91ECdThq58cZAzB9tPxG18g/view?usp=sharing

Base2 - Ratings: https://drive.google.com/file/d/1C_T1w8fc7Oa8MeTo4LMTEcv90IfEOS-6/view?usp=sharing

## Instalação das Libs e imports

In [2]:
!wget -q https://downloads.apache.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

tar: spark-3.0.1-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=974ed8b1630b371aeb63419c91e68803e5e2fd519fdb8c7f5ead237e24b2335a
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [261]:
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = '\
    --driver-memory 4G \
    --executor-memory 4G \
    pyspark-shell'

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from datetime import datetime

In [262]:
spark = SparkSession.builder\
    .master("local[*]")\
    .getOrCreate()
spark

## Leitura dos arquivos csv

In [263]:
movies_df = spark.read.csv(f'data/movies.csv', header=False, inferSchema=True, sep=';').toDF('Movie_Id', 'Title')
movies_df.show(5)

+--------+--------------------+
|Movie_Id|               Title|
+--------+--------------------+
|       1|(Dinosaur Planet,...|
|       2|(Isle of Man TT 2...|
|       3|   (Character, 1997)|
|       4|(Paula Abdul's Ge...|
|       5|(The Rise and Fal...|
+--------+--------------------+
only showing top 5 rows



In [264]:
movies_df.printSchema()

root
 |-- Movie_Id: integer (nullable = true)
 |-- Title: string (nullable = true)



In [265]:
Sratings_df = spark.read.csv(f'data/customers_rating.csv', header=True, inferSchema=True, sep=';')
ratings_df = ratings_df.na.drop("any")
ratings_df.show(5)

+-------+------+----------+--------+
|Cust_Id|Rating|      Date|Movie_Id|
+-------+------+----------+--------+
|1488844|   3.0|2005-09-06|       1|
| 822109|   5.0|2005-05-13|       1|
| 885013|   4.0|2005-10-19|       1|
|  30878|   4.0|2005-12-26|       1|
| 823519|   3.0|2004-05-03|       1|
+-------+------+----------+--------+
only showing top 5 rows



In [266]:
ratings_df.printSchema()

root
 |-- Cust_Id: integer (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Movie_Id: integer (nullable = true)



In [267]:
ratings_df.count()

41775

# Exercícios

## 1. Quantos filmes estão disponíveis no dataset?


In [268]:
print("Total de filmes:", movies_df.distinct().count())

Total de filmes: 4499


## 2. Qual é o nome dos 5 filmes com melhor média de avaliação?


In [269]:
%%time
df = ratings_df.groupby('Movie_Id')\
    .mean()\
    .select(['Movie_Id','avg(Rating)' ])\
    .withColumnRenamed('avg(Rating)', 'Média')\
    .orderBy(F.desc('Média'))\
    .limit(10)\
    .cache() # Para salvar o df computado na memória RAM. Sai imediatamente.
df.show(5)

+--------+------------------+
|Movie_Id|             Média|
+--------+------------------+
|      13|             4.552|
|       5|3.9192982456140353|
|      18| 3.784698713608666|
|       1| 3.749542961608775|
|       3|3.6411530815109345|
+--------+------------------+
only showing top 5 rows

CPU times: user 38.3 ms, sys: 7.5 ms, total: 45.7 ms
Wall time: 4.67 s


In [270]:
print('Filmes com melhores avaliações')
df.toPandas()\
    .merge(
        movies_df.select('Movie_Id', 'title').toPandas(), 
        on='Movie_Id', 
        how='inner'
    ).head(5)


Filmes com melhores avaliações


,Movie_Id,Média,title
0,13,4.552000,(Lord of the Rings: The Return of the King: Ex...
1,5,3.919298,"(The Rise and Fall of ECW, 2004)"
2,18,3.784699,"(Immortal Beloved, 1994)"
3,1,3.749543,"(Dinosaur Planet, 2003)"
4,3,3.641153,"(Character, 1997)"


## 3. Quais os 9 anos com menos lançamentos de filmes?


In [278]:
# Split na coluna de titulo para separar nome do filme e ano, foi necessario quebrar porque alguns filmes tem anos no nome
# ex. 2090;(Cleopatra 2525: The Complete Series, 2000)
# ex. 580;(THX 1138: Special Edition, 1971)

split_cols = F.split(movies_df["title"], ',')

movies_df = movies_df.withColumn('Filme', split_cols.getItem(0)) \
    .withColumn('Ano', split_cols.getItem(1))

movies_df.show()

+--------+--------------------+----------+--------------------+------+
|Movie_Id|               Title|Lancamento|               Filme|   Ano|
+--------+--------------------+----------+--------------------+------+
|       1|(Dinosaur Planet,...|      2003|    (Dinosaur Planet| 2003)|
|       2|(Isle of Man TT 2...|      2004|(Isle of Man TT 2...| 2004)|
|       3|   (Character, 1997)|      1997|          (Character| 1997)|
|       4|(Paula Abdul's Ge...|      1994|(Paula Abdul's Ge...| 1994)|
|       5|(The Rise and Fal...|      2004|(The Rise and Fal...| 2004)|
|       6|        (Sick, 1997)|      1997|               (Sick| 1997)|
|       7|       (8 Man, 1992)|      1992|              (8 Man| 1992)|
|       8|(What the #$*! Do...|      2004|(What the #$*! Do...| 2004)|
|       9|(Class of Nuke 'E...|      1991|(Class of Nuke 'E...| 1991)|
|      10|     (Fighter, 2001)|      2001|            (Fighter| 2001)|
|      11|(Full Frame: Docu...|      1999|(Full Frame: Docu...| 1999)|
|     

In [279]:
regex = "\d{4}"
movies_df = movies_df.withColumn("date_col", F.regexp_extract(movies_df["Ano"], regex, 0)) 
movies_df = movies_df.withColumn("date_col", F.to_date(movies_df["date_col"])) 
movies_df = movies_df.withColumn("Lancamento", F.year(movies_df["date_col"]))
movies_df = movies_df.drop('date_col', 'Filme', 'Ano')
movies_df.show(5)

movies_df.printSchema()

+--------+--------------------+----------+
|Movie_Id|               Title|Lancamento|
+--------+--------------------+----------+
|       1|(Dinosaur Planet,...|      2003|
|       2|(Isle of Man TT 2...|      2004|
|       3|   (Character, 1997)|      1997|
|       4|(Paula Abdul's Ge...|      1994|
|       5|(The Rise and Fal...|      2004|
+--------+--------------------+----------+
only showing top 5 rows

root
 |-- Movie_Id: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Lancamento: integer (nullable = true)



In [280]:
print('Anos com menos lancamentos:')
movies_df\
  .groupby('Lancamento')\
  .count()\
  .orderBy(F.asc('count'))\
  .show(9)

Anos com menos lancamentos:
+----------+-----+
|Lancamento|count|
+----------+-----+
|      1926|    1|
|      1922|    1|
|      1917|    1|
|      1915|    1|
|      1924|    2|
|      1918|    2|
|      1916|    2|
|      1931|    2|
|      1929|    2|
+----------+-----+
only showing top 9 rows



## 4. Quantos filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os filmes avaliados na última data de avaliação do dataset?


## 5. Dos filmes encontrados na questão anterior, quais são os 10 filmes com as piores notas e quais as notas?


## 6. Quais os id's dos 5 customers que mais avaliaram filmes e quantas avaliações cada um fez?